### Imports


In [26]:
from code_analysis import CFG, CFGReader
import queue
import time
import numpy as np

In [27]:
def get_nodes(cfg : CFG, node_type: str, image: str = None):
    return [node for node in cfg.get_node_ids() if cfg.get_type(node) == node_type and (image is None or cfg.get_image(node) == image)]

In [28]:
def PatternTraversalFlowAnalysis(cfg : CFG, node_set, mode = "reaching") :
    assert mode in ["reaching", "reachable"]
    """_summary_

    Args:
        cfg (CFG): _description_
        node_set (_type_): _description_
        start_node (_type_): entry or exit depending on the algorithm

    Returns:
        _type_: _description_
    """
    start_node = 'Entry' if mode == "reaching" else 'Exit'
    
    start_nodes = get_nodes(cfg, start_node)
    
    workList = queue.LifoQueue()
    
    IN = {node: True for node in cfg.get_node_ids()}
    OUT = {node: True for node in cfg.get_node_ids()}
    
    visited = set(  )
    if mode == "reaching":
        for node in start_nodes:
            IN[node] = False
    elif mode == "reachable":
        for node in start_nodes:
            OUT[node] = False
    
    
    for node in start_nodes:
        visited.add(node)
    for node in start_nodes:
        workList.put(node)

    while not workList.empty() :
        
        node = workList.get()
        if mode == "reaching":
            OUT[node] = node in node_set or IN[node]
        elif mode == "reachable":
            IN[node] = node in node_set or OUT[node]
        
        next = cfg.get_children(node) if mode == "reaching" else cfg.get_parents(node)
        if cfg.get_type(node) == "CallBegin" and mode == "reaching":
            next.append(node +1)
        if cfg.get_type(node) == "CallEnd" and mode == "reachable":
            next.append(node -1)
        for new_node in next:
            propagate_flag = (OUT[node] < IN[new_node]) or (new_node not in visited) if mode == "reaching" else \
                            (IN[node] < OUT[new_node]) or (new_node not in visited)
            if propagate_flag :
                if mode == "reaching":
                    IN[new_node] = OUT[node]
                elif mode == "reachable":
                    OUT[new_node] = IN[node]
                workList.put(new_node)
                visited.add(new_node)

    return IN, OUT
    
    

### Performances

In [29]:
reader = CFGReader()
graph1 = reader.read_cfg("../tp/perf/graph1.cfg.json")

S = get_nodes(graph1, "Pattern")
print(S)


i,o = PatternTraversalFlowAnalysis(graph1, S, "reachable")

print(i)
print(o)


[22, 53, 58, 72, 82]
{0: False, 1: False, 2: False, 3: False, 4: False, 5: False, 6: False, 7: False, 8: False, 9: False, 10: False, 11: False, 12: False, 13: False, 14: False, 15: False, 16: False, 17: False, 18: False, 19: False, 20: False, 21: False, 22: True, 23: False, 24: False, 25: False, 26: False, 27: False, 28: False, 29: False, 30: False, 31: False, 32: False, 33: False, 34: False, 35: False, 36: False, 37: False, 38: False, 39: False, 40: False, 41: False, 42: False, 43: False, 44: False, 45: False, 46: False, 47: False, 48: False, 49: False, 50: False, 51: False, 52: False, 53: True, 54: False, 55: False, 56: False, 57: True, 58: True, 59: False, 60: False, 61: False, 62: False, 63: False, 64: False, 65: False, 66: False, 67: True, 68: False, 69: False, 70: False, 71: False, 72: True, 73: False, 74: False, 75: False, 76: False, 77: False, 78: False, 79: False, 80: False, 81: False, 82: True, 83: False, 84: True, 85: False, 86: False, 87: False, 88: False, 89: False, 90: Fa

In [30]:
def read_one_file(file,type="Pattern", mode = "reachable"):
    reader = CFGReader()
    graph = reader.read_cfg(file)
    #calculate nb node + nb edges
    nb_nodes = len(graph.get_node_ids())
    nb_edges = sum([len(graph.get_children(node)) for node in graph.get_node_ids()])
    print(f"Nb nodes: {nb_nodes}, Nb edges: {nb_edges}, Total: {nb_nodes + nb_edges}")
    
    S = get_nodes(graph, type)
    time1 = time.time()
    i,o = PatternTraversalFlowAnalysis(graph, S, mode)
    time2 = time.time()
    print(f"Time elapsed: {time2 - time1}")


for i in range(1,5):
    read_one_file(f"../tp/perf/graph{i}.cfg.json")
    
    

Nb nodes: 100, Nb edges: 144, Total: 244
Time elapsed: 0.0001862049102783203
Nb nodes: 1000, Nb edges: 1509, Total: 2509
Time elapsed: 0.0019159317016601562
Nb nodes: 10000, Nb edges: 15267, Total: 25267
Time elapsed: 0.04728889465332031
Nb nodes: 100000, Nb edges: 152895, Total: 252895
Time elapsed: 3.1771039962768555


## f-open f-close

In [31]:
##get the nodes for f-open and f-close
reader = CFGReader()
file = "../tp/part_1/file1.php.cfg.json"
graph = reader.read_cfg(file)


S = get_nodes(graph, "FunctionCall", "fclose")
i,o = PatternTraversalFlowAnalysis(graph, S, "reachable")
fopen = get_nodes(graph, "FunctionCall", "fopen")

for node in fopen:
    if not i[node] :
        line = graph.get_position(node)
        print(f"Node {node} is a fopen that is not closed  at line {line}")
    else:
        print(f"Node {node} is a fopen that is closed")


Node 11 is a fopen that is not closed  at line None


### Protection

Ici nous souhaitons voir si tout appel à la base de donnée est précédé d'une instruction if(has_cap('use_db'))
Nous procederons donc ainsi :
- identification des noeuds de requete sql dans le cfg
- identification des groupes de noeuds comportant la verification if(has_cap('use_db')) dans le cfg
- recherche des noeuds "definitly reaching" ces requetes et pour chaque requete et check si un groupe de noeud des if(has_cap..) est contenu dedans

In [32]:
cfg_reader = CFGReader()
file_cfg = "../tp/part_2/wp-db.php.cfg.json"



In [33]:

graph_cfg = cfg_reader.read_cfg(file_cfg)


### Here we find out the nodes ids that contains sql query

In [58]:
def find_execute_method(cfg : CFG, node_id) :
    if cfg.get_type(node_id) == "MethodCall" and cfg.get_image(node_id) == "execute":
        return True
    return False

def find_sql_query(cfg : CFG, node_id) :
    if cfg.get_type(node_id) == "Id" and cfg.get_image(node_id) in ["mysql_query","mysqli_query"] :
        return True
    return False
sql_nodes_ids = [node_id for node_id in graph_cfg.get_node_ids() if find_execute_method(graph_cfg, node_id) or find_sql_query(graph_cfg, node_id) ]
      

In [59]:
for id in sql_nodes_ids :
    print(graph_cfg.get_type(id))
    print(graph_cfg.get_image(id))
    print("##############")
    


Id
mysql_query
##############
Id
mysql_query
##############
Id
mysql_query
##############


#### now lets find nodes ids that contains the if(has_cap("use_db"))

In [74]:
def has_cap_nodes(cfg : CFG, id : int) :
    #return the id of has cap right child after the condition if it is a has cap or -1
    if cfg.get_type(id) == "If" :
        child = cfg.get_children(id)[0]
        if cfg.get_type( child) == "FunctionCall" and cfg.get_image(child) == "has_cap" :
            #go down two times to see if argument List is "use_db"
            arg = cfg.get_children(cfg.get_children(cfg.get_children(child)[0])[0])[0]
            if cfg.get_type(arg) == "StringLiteral" and cfg.get_image(arg) == "use_db" :
                #go to the condition node and get the right child id
                node_id = arg
                while cfg.get_type(node_id) != "Condition" :
                    node_id+=1
                return cfg.get_children(node_id)[1]
    return -1
                
            

In [75]:
hascap_nodes_ids = [has_cap_nodes(graph_cfg,node_id) for node_id in graph_cfg.get_node_ids() if has_cap_nodes(graph_cfg, node_id) != -1 ]
print(hascap_nodes_ids)

[533, 651]


now we compute a definitly reaching ptfa on the hascap_nodes_ids to see which nodes are definitly reached by these
and correlate with sql query nodes

In [80]:
i,o = PatternTraversalFlowAnalysis(graph_cfg, hascap_nodes_ids, "reachable")
for node_id in sql_nodes_ids :
    if not i[node_id]:
        print(f"the sql query at node {node_id} if not following a if has_cap(\"use_db\") True condition")

the sql query at node 631 if not following a if has_cap("use_db") True condition
the sql query at node 680 if not following a if has_cap("use_db") True condition
